In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.5.0
sys.version_info(major=3, minor=8, micro=5, releaselevel='final', serial=0)
matplotlib 3.4.2
numpy 1.19.5
pandas 1.3.0
sklearn 0.24.2
tensorflow 2.5.0
tensorflow.keras 2.5.0


In [2]:
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
print(housing.DESCR)
print(housing.data.shape)
print(housing.target.shape)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block
        - HouseAge      median house age in block
        - AveRooms      average number of rooms
        - AveBedrms     average number of bedrooms
        - Population    block population
        - AveOccup      average house occupancy
        - Latitude      house block latitude
        - Longitude     house block longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
http://lib.stat.cmu.edu/datasets/

The target variable is the median house value for California districts.

This dataset was derived from the 1990 U.S. census, using one row per census
block group. A block group is the smallest geographical unit for which the U.S.
Census Bur

In [3]:
from sklearn.model_selection import train_test_split

x_train_all, x_test, y_train_all, y_test = train_test_split(
    housing.data, housing.target, random_state = 7)
x_train, x_valid, y_train, y_valid = train_test_split(
    x_train_all, y_train_all, random_state = 11)
print(x_train.shape, y_train.shape)
print(x_valid.shape, y_valid.shape)
print(x_test.shape, y_test.shape)


(11610, 8) (11610,)
(3870, 8) (3870,)
(5160, 8) (5160,)


In [4]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_valid_scaled = scaler.transform(x_valid)
x_test_scaled = scaler.transform(x_test)

In [5]:
# RandomizedSearchCV
# 1. 转化为sklearn的model
# 2. 定义超参数
# 3. 搜索参数
def build_model(hidden_layers = 1, layer_size = 30, learning_rate = 3e-3):
    model = keras.models.Sequential()
    model.add(keras.layers.Dense(layer_size, activation='relu',
                                input_shape=x_train.shape[1:]))
    for _ in range(hidden_layers - 1):
        model.add(keras.layers.Dense(layer_size,
                                    activation='relu'))
    model.add(keras.layers.Dense(1))
    optimizer = keras.optimizers.SGD(learning_rate)
    model.compile(loss="mse", optimizer=optimizer)
    return model

callbacks = [keras.callbacks.EarlyStopping(
    patience=5, min_delta=1e-2)]
sklearn_model = keras.wrappers.scikit_learn.KerasRegressor(build_model)

learning_rate = [1e-4, 3e-4, 1e-3, 3e-3, 1e-2, 3e-2]


Epoch 1/10


2021-07-31 09:12:33.597719: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-07-31 09:12:33.720184: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)


363/363 [==============================] - 1s 2ms/step - loss: 1.1511 - val_loss: 0.7071
Epoch 2/10
363/363 [==============================] - 0s 1ms/step - loss: 0.6244 - val_loss: 0.6373
Epoch 3/10
363/363 [==============================] - 0s 1ms/step - loss: 0.5682 - val_loss: 0.5893
Epoch 4/10
363/363 [==============================] - 0s 1ms/step - loss: 0.5306 - val_loss: 0.5550
Epoch 5/10
363/363 [==============================] - 1s 1ms/step - loss: 0.5129 - val_loss: 0.5285
Epoch 6/10
363/363 [==============================] - 1s 1ms/step - loss: 0.4961 - val_loss: 0.5113
Epoch 7/10
363/363 [==============================] - 1s 2ms/step - loss: 0.4740 - val_loss: 0.4970
Epoch 8/10
363/363 [==============================] - 1s 2ms/step - loss: 0.4642 - val_loss: 0.4858
Epoch 9/10
363/363 [==============================] - 0s 1ms/step - loss: 0.4536 - val_loss: 0.4787
Epoch 10/10
363/363 [==============================] - 1s 2ms/step - loss: 0.4474 - val_loss: 0.4700


In [14]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import reciprocal
# f(x) = 1(x*log(b/a)) a<= x <=b
param_gird=[{
    'hidden_layers':[1,2,3,4],
    'layer_size':[i for i in  range(1,101)],
    'learning_rate':reciprocal(1e-4,1e-2)
}]
cv = RandomizedSearchCV(sklearn_model,param_gird,n_iter=10)

In [12]:
RandomizedSearchCV?

In [15]:
histories = cv.fit(x_train_scaled, y_train, epochs=10, 
                  validation_data=(x_valid_scaled, y_valid),
                 callbacks = callbacks)

Epoch 1/10
291/291 [==============================] - 2s 4ms/step - loss: 3.3007 - val_loss: 2.1510
Epoch 2/10
291/291 [==============================] - 1s 3ms/step - loss: 1.6517 - val_loss: 1.5259
Epoch 3/10
291/291 [==============================] - 1s 2ms/step - loss: 1.3753 - val_loss: 1.4119
Epoch 4/10
291/291 [==============================] - 1s 2ms/step - loss: 1.3293 - val_loss: 1.3899
Epoch 5/10
291/291 [==============================] - 1s 3ms/step - loss: 1.3218 - val_loss: 1.3848
Epoch 6/10
291/291 [==============================] - 1s 3ms/step - loss: 1.3207 - val_loss: 1.3835
Epoch 7/10
291/291 [==============================] - 1s 2ms/step - loss: 1.3204 - val_loss: 1.3828
Epoch 8/10
291/291 [==============================] - 1s 3ms/step - loss: 1.3204 - val_loss: 1.3827
Epoch 9/10
73/73 [==============================] - 0s 2ms/step - loss: 1.2872
Epoch 1/10
291/291 [==============================] - 1s 3ms/step - loss: 3.2659 - val_loss: 2.1662
Epoch 2/10
291/291 [=

291/291 [==============================] - 1s 2ms/step - loss: 0.8622 - val_loss: 0.6058
Epoch 2/10
291/291 [==============================] - 1s 2ms/step - loss: 0.5299 - val_loss: 0.4753
Epoch 3/10
291/291 [==============================] - 0s 2ms/step - loss: 0.4180 - val_loss: 0.4219
Epoch 4/10
291/291 [==============================] - 0s 1ms/step - loss: 0.3890 - val_loss: 0.4037
Epoch 5/10
291/291 [==============================] - 0s 1ms/step - loss: 0.3741 - val_loss: 0.3929
Epoch 6/10
291/291 [==============================] - 1s 2ms/step - loss: 0.3628 - val_loss: 0.4169
Epoch 7/10
291/291 [==============================] - 0s 1ms/step - loss: 0.3566 - val_loss: 0.3763
Epoch 8/10
291/291 [==============================] - 0s 1ms/step - loss: 0.3493 - val_loss: 0.3999
Epoch 9/10
291/291 [==============================] - 0s 1ms/step - loss: 0.3438 - val_loss: 0.3591
Epoch 10/10
73/73 [==============================] - 0s 1ms/step - loss: 0.3480
Epoch 1/10
291/291 [===========

291/291 [==============================] - 0s 1ms/step - loss: 0.8255 - val_loss: 0.8974
Epoch 9/10
291/291 [==============================] - 0s 1ms/step - loss: 0.7937 - val_loss: 0.8665
Epoch 10/10
73/73 [==============================] - 0s 669us/step - loss: 0.7142
Epoch 1/10
291/291 [==============================] - 1s 1ms/step - loss: 5.4063 - val_loss: 4.6428
Epoch 2/10
291/291 [==============================] - 0s 1ms/step - loss: 3.6275 - val_loss: 3.3491
Epoch 3/10
291/291 [==============================] - 0s 1ms/step - loss: 2.7288 - val_loss: 2.6245
Epoch 4/10
291/291 [==============================] - 0s 1ms/step - loss: 2.2033 - val_loss: 2.1788
Epoch 5/10
291/291 [==============================] - 0s 1ms/step - loss: 1.8596 - val_loss: 1.8558
Epoch 6/10
291/291 [==============================] - 0s 1ms/step - loss: 1.5969 - val_loss: 1.5942
Epoch 7/10
291/291 [==============================] - 0s 1ms/step - loss: 1.3830 - val_loss: 1.3766
Epoch 8/10
291/291 [=========

291/291 [==============================] - 0s 2ms/step - loss: 0.6114 - val_loss: 0.6544
Epoch 5/10
291/291 [==============================] - 0s 1ms/step - loss: 0.5788 - val_loss: 0.6213
Epoch 6/10
291/291 [==============================] - 0s 1ms/step - loss: 0.5518 - val_loss: 0.5927
Epoch 7/10
291/291 [==============================] - 0s 1ms/step - loss: 0.5289 - val_loss: 0.5750
Epoch 8/10
291/291 [==============================] - 0s 1ms/step - loss: 0.5099 - val_loss: 0.5471
Epoch 9/10
291/291 [==============================] - 0s 1ms/step - loss: 0.4937 - val_loss: 0.5294
Epoch 10/10
73/73 [==============================] - 0s 772us/step - loss: 0.5059
Epoch 1/10
291/291 [==============================] - 1s 2ms/step - loss: 1.6224 - val_loss: 0.8762
Epoch 2/10
291/291 [==============================] - 0s 1ms/step - loss: 0.7132 - val_loss: 0.7372
Epoch 3/10
291/291 [==============================] - 0s 1ms/step - loss: 0.6452 - val_loss: 0.6795
Epoch 4/10
291/291 [=========

73/73 [==============================] - 0s 2ms/step - loss: 0.3892
Epoch 1/10
291/291 [==============================] - 1s 3ms/step - loss: 1.7449 - val_loss: 0.8280
Epoch 2/10
291/291 [==============================] - 1s 2ms/step - loss: 0.6412 - val_loss: 0.5983
Epoch 3/10
291/291 [==============================] - 1s 2ms/step - loss: 0.5262 - val_loss: 0.5412
Epoch 4/10
291/291 [==============================] - 1s 2ms/step - loss: 0.4834 - val_loss: 0.4986
Epoch 5/10
291/291 [==============================] - 1s 2ms/step - loss: 0.4582 - val_loss: 0.4735
Epoch 6/10
291/291 [==============================] - 1s 3ms/step - loss: 0.4430 - val_loss: 0.4623
Epoch 7/10
291/291 [==============================] - 1s 2ms/step - loss: 0.4320 - val_loss: 0.4465
Epoch 8/10
291/291 [==============================] - 1s 2ms/step - loss: 0.4243 - val_loss: 0.4373
Epoch 9/10
291/291 [==============================] - 1s 2ms/step - loss: 0.4182 - val_loss: 0.4369
Epoch 10/10
73/73 [=============

291/291 [==============================] - 0s 1ms/step - loss: 0.3977 - val_loss: 0.4174
Epoch 8/10
291/291 [==============================] - 0s 1ms/step - loss: 0.3895 - val_loss: 0.4001
Epoch 9/10
291/291 [==============================] - 0s 1ms/step - loss: 0.3809 - val_loss: 0.3979
Epoch 10/10
73/73 [==============================] - 0s 763us/step - loss: 0.3887
Epoch 1/10
291/291 [==============================] - 1s 1ms/step - loss: 1.3644 - val_loss: 0.5864
Epoch 2/10
291/291 [==============================] - 0s 1ms/step - loss: 0.5028 - val_loss: 0.4892
Epoch 3/10
291/291 [==============================] - 0s 1ms/step - loss: 0.4387 - val_loss: 0.4511
Epoch 4/10
291/291 [==============================] - 0s 1ms/step - loss: 0.4157 - val_loss: 0.4246
Epoch 5/10
291/291 [==============================] - 0s 966us/step - loss: 0.4001 - val_loss: 0.4152
Epoch 6/10
291/291 [==============================] - 0s 1ms/step - loss: 0.3884 - val_loss: 0.4039
Epoch 7/10
291/291 [=======

291/291 [==============================] - 0s 1ms/step - loss: 0.5482 - val_loss: 0.5731
Epoch 4/10
291/291 [==============================] - 0s 1ms/step - loss: 0.5150 - val_loss: 0.5403
Epoch 5/10
291/291 [==============================] - 0s 1ms/step - loss: 0.4914 - val_loss: 0.5174
Epoch 6/10
291/291 [==============================] - 0s 1ms/step - loss: 0.4739 - val_loss: 0.5058
Epoch 7/10
291/291 [==============================] - 0s 1ms/step - loss: 0.4624 - val_loss: 0.4870
Epoch 8/10
291/291 [==============================] - 0s 1ms/step - loss: 0.4515 - val_loss: 0.4740
Epoch 9/10
291/291 [==============================] - 0s 1ms/step - loss: 0.4441 - val_loss: 0.4653
Epoch 10/10
73/73 [==============================] - 0s 628us/step - loss: 0.4367
Epoch 1/10
291/291 [==============================] - 1s 1ms/step - loss: 1.4321 - val_loss: 0.8054
Epoch 2/10
291/291 [==============================] - 0s 1ms/step - loss: 0.6660 - val_loss: 0.6613
Epoch 3/10
291/291 [=========

In [19]:
print(cv.best_params_)
model2 = cv.best_estimator_.model
model2.evaluate(x_test_scaled,y_test)

{'hidden_layers': 3, 'layer_size': 91, 'learning_rate': 0.006910864599504031}
162/162 [==============================] - 0s 824us/step - loss: 0.3601


0.3601077198982239